In [1]:
import h5py    
import numpy as np    
import torch
import gc
import csv
from torch import nn
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import random
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
from NN_models import NN_2_256, NN_8_256, NN_8_64


def ref(x, y):
    ''' 
    returns reference energies for points of a reaction grid from Reference_data.csv
    '''
    hartree2kcal = 627.5095
    with open("Reference_data.csv", newline='', encoding='cp1251') as csvfile:
        ref_file = csv.reader(csvfile, delimiter=",")
        k = 1
        if y == 391:
            k = hartree2kcal
        ref = []
        for n, i in enumerate(ref_file):
            if x <= n + 1 <= y:
                ref.append((i[0], float(i[2]) * k))

        return ref

def load_ref_energies():
    '''Returns {db_name: [equation, energy]}'''
    ref_e = { # Получение референсных энергий
        "MGAE109":ref(8, 116),
        "IP13":ref(155, 167),
        "EA13":ref(180, 192),
        "PA8":ref(195, 202),
        "DBH76":ref(251, 288) + ref(291, 328),
        "NCCE31":ref(331, 361),
        "ABDE4":ref(206, 209),
        # "AE17":ref(375, 391),
        "pTC13":ref(232, 234) + ref(237, 241) + ref(244, 248)
        } 
    return ref_e

def load_component_names():
    '''
    Returns {db_name: {id: {'Components': [...], 'Coefficients: [...]'
                                }
                            }
                        }
     which is a dictionary with Components and Coefficients data about all reactions
    '''
    with open("total_dataframe_sorted_final.csv", newline='', encoding='cp1251') as csvfile:
        ref_file = csv.reader(csvfile, delimiter=",")
        ref = dict()
        current_database = None
        
        for n, line in enumerate(ref_file):
            line = np.array(line)
            if n == 0:
                components = np.array(line)
            else:
                reaction_id = int(line[0])
                reaction_database = line[1]
                reaction_component_num = np.nonzero(list(map(float, line[2:])))[0] + 2
                if reaction_database in ref:
                    ref[reaction_database][reaction_id] = {'Components': components[reaction_component_num], 'Coefficients': line[reaction_component_num]}
                else: 
                    ref[reaction_database] = {reaction_id: {'Components': components[reaction_component_num], 'Coefficients': line[reaction_component_num]}}
        return ref
    
    
def get_compounds_coefs_energy_v2(reactions, energies):
    '''Returns {id: 
                    {'Components': [...], 'Coefficients: [...]', 'Energy: float', Database: str
                                }
                            }
    which is a dictionaty from load_component_names with Energy information added
    '''
    data_final = dict()
    i = 0
    databases = load_ref_energies().keys()
    for database in databases:
        data = reactions[database]
        for reaction in data:
            data_final[i] = {'Database': database,
                             'Components': reactions[database][reaction]['Components'], #.astype(object),
                             'Coefficients': torch.Tensor(reactions[database][reaction]['Coefficients'].astype(np.float32)),
                             'Energy': torch.Tensor(np.array(energies[database][reaction][1]))
            
        }
            i += 1
        
    return data_final


def get_h5_names(reaction):
    '''reaction must be from the function get_compounds_coefs_energy_v2'''
    database_match = {
        'MGAE109': 'mgae109',
        'IP13': 'ip13',
        'EA13': 'ea13',
        'PA8': 'pa8',
        'DBH76': 'ntbh38',
        'NCCE31': 'ncce31',
        'ABDE4': 'abde4',
        'AE17': 'ae17',
        'pTC13': 'ptc13'
    }
    names = []
    for elem in reaction['Components']:
        database = database_match[reaction['Database']]
        names.append(f'{elem}.h5')
    return names


def add_reaction_info_from_h5(reaction):
    '''
    reaction must be from get_compounds_coefs_energy_v2
    returns merged descriptos array X, integration weights, 
    a and b densities and indexes for backsplitting
    
    Adds the following information to the reaction dict using h5 files from the dataset:
    Grid : np.array with grid descriptors
    Weights : list with integration weights of grid points
    Densities : np.array with alpha and beta densities data for grid points
    HF_energies : list of Total HF energy (T+V) which needs to be added to E_xc
    backsplit_ind: list of indexes where we concatenate molecules' grids
    '''
    X = np.array([])
    backsplit_ind = []
    HF_energies = np.array([])
    for component_filename in get_h5_names(reaction):
        with h5py.File(f'data/{component_filename}', "r") as f:
            HF_energies = np.append(HF_energies, f["ener"][:][0])
            X_raw = np.array(f["grid"][:])
            if len(X) == 0:
                X = X_raw[:, 3:-1]
            else:
                X = np.vstack((X, X_raw[:, 3:-1]))
            backsplit_ind.append(len(X))
    densities = X[:, 1:3]
    weights = X[:,0]
    X = X[:, 1:]
    # sigma_a_b to norm_grad=sigma_a + sigma_b + 2*sigma_a_b
    X[:, 3] = X[:, 2] + X[:, 4] + 2*X[:, 3]
    # log grid data
    eps = 10**(-12)
    X = np.log(X+eps)
    
    labels = ['Grid', 'Weights', 'Densities', 'HF_energies', 'backsplit_ind']
    values = [X, weights, densities, HF_energies, backsplit_ind]
    for label, value in zip(labels, values):
        reaction[label] = torch.Tensor(value)

    return reaction


def make_reactions_dict():
    '''
    Returns a dict like {reaction_id: {*reaction info}} with all info available listed below:
    ['Database', 'Components', 'Coefficients', 'Energy', 'Grid', 'Weights', 'Densities', 'HF_energies', 'backsplit_ind']
    '''
    data = get_compounds_coefs_energy_v2(load_component_names(), load_ref_energies())
    for i in data.keys():
        data[i] = add_reaction_info_from_h5(data[i])

    return data


def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(42)

In [2]:
%%time

data = make_reactions_dict()

CPU times: user 10.5 s, sys: 5.37 s, total: 15.9 s
Wall time: 16.3 s


In [3]:
data[0]

{'Database': 'MGAE109',
 'Components': array(['C_mgae109', 'H_mgae109', 'CH_mgae109'], dtype='<U20'),
 'Coefficients': tensor([ 1.,  1., -1.]),
 'Energy': tensor(84.2300),
 'Grid': tensor([[  4.1464,   4.1465,   0.6599,  ...,   0.6602,   2.0153,  -5.5657],
         [  4.1464,   4.1465,   5.0959,  ...,   5.0962,   2.0569,  -1.1297],
         [  4.1463,   4.1465,   7.6894,  ...,   7.6897,   2.4699,   1.4638],
         ...,
         [ -7.6264,  -8.0338, -13.8937,  ..., -14.5039,  -8.2339,  -8.5278],
         [ -7.3618,  -7.7324, -13.3484,  ..., -13.8880,  -7.9495,  -8.2130],
         [ -7.6264,  -8.0338, -13.8937,  ..., -14.5039,  -8.2339,  -8.5278]]),
 'Weights': tensor([2.5717e-17, 9.9780e-15, 3.2611e-13,  ..., 2.6018e-02, 2.2653e-02,
         2.6018e-02]),
 'Densities': tensor([[6.3204e+01, 6.3213e+01],
         [6.3204e+01, 6.3213e+01],
         [6.3201e+01, 6.3210e+01],
         ...,
         [4.8742e-04, 3.2433e-04],
         [6.3506e-04, 4.3839e-04],
         [4.8742e-04, 3.2433e-0

In [4]:
y_single = [0.0310907, 0.01554535, 
            3.72744,   7.06042,
            12.9352,   18.0578,
            -0.10498,  -0.32500,
            0.0310907,  0.01554535,  -1/(6*np.pi**2),
            13.0720,    20.1231,      1.06835,
            42.7198,   101.578,      11.4813,
            -0.409286,  -0.743294,   -0.228344,
            1]

nconstants = len(y_single)

device = torch.device('cuda:0') if torch.cuda.is_available else torch.device('cpu')


lst = []
for i in range(len(data)):
    lst.append(data[i]['Grid'])
    
all_grid_data = torch.cat(lst)
print(all_grid_data.shape)

torch.Size([88862460, 7])


In [5]:
%%time
stdscaler = StandardScaler()
stdscaler.fit(np.array(all_grid_data))

CPU times: user 10.8 s, sys: 1.73 s, total: 12.6 s
Wall time: 12.6 s


StandardScaler()

In [6]:
%%time
for i in range(len(data)):
    data[i]['Grid'] = torch.Tensor(stdscaler.transform(data[i]['Grid']))

CPU times: user 3.02 s, sys: 858 ms, total: 3.88 s
Wall time: 3.9 s


In [7]:
data[0]['Grid']

tensor([[ 1.6719,  1.5777,  1.2155,  ...,  1.2445,  1.3427,  0.3330],
        [ 1.6719,  1.5777,  1.6793,  ...,  1.6905,  1.3486,  0.8961],
        [ 1.6719,  1.5776,  1.9504,  ...,  1.9512,  1.4078,  1.2254],
        ...,
        [-0.0775, -0.0087, -0.3059,  ..., -0.2801, -0.1262, -0.0431],
        [-0.0381,  0.0306, -0.2489,  ..., -0.2182, -0.0854, -0.0031],
        [-0.0775, -0.0087, -0.3059,  ..., -0.2801, -0.1262, -0.0431]])

In [40]:
%%time
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):

        self.data = data
        
    def __getitem__(self, i):
        self.data[i].pop('Database', None)

        return self.data[i], y_single
    
    def __len__(self):
        return len(self.data.keys())


train_set = Dataset(data=data)
train_dataloader = torch.utils.data.DataLoader(train_set,
                                               batch_size=None,
                                               num_workers=1,
                                               pin_memory=True,
                                               shuffle=True)

criterion = nn.MSELoss()

CPU times: user 287 µs, sys: 0 ns, total: 287 µs
Wall time: 299 µs


In [9]:
# np.frombuffer(X_batch['Components'][0], dtype='<U20')

In [10]:
# torch.cuda.reset()
# torch.cuda.empty_cache()
# del X_batch, y_batch

In [11]:
# from GPUtil import showUtilization as gpu_usage

In [65]:
def train(model, criterion, optimizer, scheduler, train_dataloader, n_epochs=2):
    
    torch.set_printoptions(precision=5)
    
    train_loss_mse = []
    train_loss_mae = []
    test_loss_mse = []
    test_loss_mae = []


    for epoch in range(n_epochs):
        print('Epoch', epoch+1)
        # train
        model.train()


        train_mse_losses_per_epoch = []
        train_mae_losses_per_epoch = []
        
        progress_bar = tqdm(train_dataloader)

        # gradscaler = GradScaler()
        for i, (X_batch, y_batch) in enumerate(progress_bar):
            # print(np.frombuffer(X_batch['Components'][0], dtype='<U20'), X_batch['Grid'][0].shape)

            X_batch = X_batch['Grid'].to(device, non_blocking=True)
            y_batch = torch.tile(torch.Tensor(y_batch), [X_batch.shape[0],1]).to(device, non_blocking=True)
            # with torch.cuda.amp.autocast(dtype=torch.bfloat16):
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            MAE = mean_absolute_error(predictions.cpu().detach(), y_batch.cpu().detach())
            MSE = loss.item()
            train_mse_losses_per_epoch.append(MSE)
            train_mae_losses_per_epoch.append(MAE)
            # scheduler.step(MAE)
            progress_bar.set_postfix(MAE = MAE, MSE = MSE)

            pred = predictions[0].cpu().detach().numpy()
            del X_batch, y_batch, predictions, loss, MAE, MSE
            gc.collect()
            torch.cuda.empty_cache()
            
        train_loss_mse.append(np.mean(train_mse_losses_per_epoch))
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))
        
        print(f'train MSE Loss = {train_loss_mse[epoch]:.8f}')
        print(f'train MAE Loss = {train_loss_mae[epoch]:.8f}')
        
    return train_loss_mse, train_loss_mae, pred

In [46]:
torch.cuda.empty_cache()

In [73]:
model = NN_2_256(DFT='SVWN').to(device)
# model.load_state_dict(torch.load('predopt/predopt_8_256.param', map_location=device))

In [87]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-2)

In [88]:
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 400, 700, 1000], gamma=1)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=100, factor=0.5)

In [1]:
train_loss_mse, train_loss_mae, preds = train(model, criterion, optimizer, 
                                              scheduler, train_dataloader, n_epochs=1)

print(train_loss_mse, train_loss_mae)
print('predicted coef', '\n', preds)
print('exact coef', '\n', np.array(y_single))

NameError: name 'train' is not defined

In [19]:
# torch.save(model.state_dict(), 'predopt/predopt_8_64_std_log.param')